In [1]:
import warnings
warnings.simplefilter("ignore")

## Escalamiento de los datos

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<div class="alert alert-success">
  Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
</div>

Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [3]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('data/X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [5]:
scaler.mean_

array([3.26856097e+07, 2.00213864e+03, 2.12655203e+00, 1.08545497e+02,
       1.02848480e+04, 4.06338081e+07, 6.44678363e+00])

In [6]:
scaler.scale_

array([4.06007486e+07, 1.20607515e+01, 7.61872544e-01, 2.35750459e+01,
       1.64204185e+04, 2.29300838e+08, 1.06820113e+00])

In [7]:
X.values

array([[4.25000000e+08, 2.00900000e+03, 1.78000000e+00, ...,
        4.83400000e+03, 2.37000000e+08, 7.90000000e+00],
       [3.06000000e+08, 2.00213073e+03, 2.12697615e+00, ...,
        1.43000000e+02, 4.04553863e+07, 7.10000000e+00],
       [3.00000000e+08, 2.00700000e+03, 2.35000000e+00, ...,
        4.83500000e+04, 3.00000000e+08, 7.10000000e+00],
       ...,
       [7.00000000e+03, 2.00500000e+03, 2.12697615e+00, ...,
        9.30000000e+01, 3.25000000e+03, 7.80000000e+00],
       [3.96700000e+03, 2.01200000e+03, 2.35000000e+00, ...,
        2.38600000e+03, 4.04553863e+07, 6.30000000e+00],
       [1.10000000e+03, 2.00400000e+03, 1.85000000e+00, ...,
        1.63000000e+02, 1.10000000e+03, 6.60000000e+00]])

In [8]:
scaler.transform(X_train)

array([[-0.79889191, -0.17732259, -0.36298988, ..., -0.54242515,
        -0.17611714,  0.14343401],
       [ 1.04220715, -0.09440902, -0.36298988, ..., -0.28652424,
         0.14987382, -1.63525723],
       [-0.5710636 ,  0.65181314,  0.29328787, ...,  0.69585023,
        -0.13359658,  0.14343401],
       ...,
       [-0.7681043 ,  0.23724527, -0.36298988, ..., -0.59717406,
        -0.17066579,  0.89235664],
       [-0.68189899,  1.066381  ,  0.29328787, ..., -0.22337116,
        -0.155402  , -1.72887256],
       [ 0.67275583, -0.34314974, -0.36298988, ..., -0.55356981,
         0.1280684 ,  0.51789533]])

In [9]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [10]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train,y_train)
model_scaled.fit(X_train_scaled,y_train)

Lasso()

In [11]:
print(model.score(X_test,y_test))
print(model_scaled.score(X_test_scaled,y_test))

0.5711076726618415
0.5711076736469591


<div class="alert alert-success">
  Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
</div>

## Simplificar las transformaciones con pipelines

<div class="alert alert-info">
  Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar <code> sklearn.pipeline.make_pipeline </code> que permite encadenar transformaciones a tus modelos.
</div>

In [12]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(),
                            Lasso())

model_scaled.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('lasso', Lasso())])

In [13]:
print(model_scaled.score(X_test,y_test))

0.5711076736469591


## Crear nuevas features de forma automática

In [14]:
A = np.arange(6).reshape(3, 2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [15]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

``PolynomialFeatures`` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [16]:
X.shape

(4104, 7)

In [17]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(4104, 36)

In [18]:
model_poly = make_pipeline(PolynomialFeatures(2),
                          Lasso())
model_poly.fit(X_train,y_train)
model_poly.score(X_test,y_test)

0.5985528746025143

In [19]:
model = Lasso()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5711076726618415

## Crear features categóricas

<div class="alert alert-success">
  En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.
</div>

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se. 

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

_**Encoding one-hot**_

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [20]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [21]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido ``pd.get_dummies``. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [22]:
movies_obj = pd.read_csv('movies_obj.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../vol/intermediate_results/movies_obj.csv'

In [ ]:
movies_obj.apply(pd.Series.nunique).sort_values()

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [ ]:
!pip install category_encoders

In [ ]:
categoricals = pd.read_csv('../vol/intermediate_results/categoricals.csv').set_index('Unnamed: 0')

In [ ]:
categoricals.head(2)

In [ ]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [ ]:
X_binenc = pd.concat([X,categoricals],axis=1)

In [ ]:
X_binenc.head()

In [ ]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [ ]:
encoder.fit_transform(X_binenc).shape

In [ ]:
X_binenc = encoder.fit_transform(X_binenc)

In [ ]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,y)

In [ ]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

In [ ]:
model_binenc = Lasso()
model = Lasso()

In [ ]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

In [ ]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

## Conocimiento experto

<div class="alert alert-success">
  Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**. <br>
  Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).** <br>
  Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.
</div>

## Más datos de calidad

Nada le gana conseguir más datos que sean encodeables en features de calidad.

_**Píramide de Maslow del Machine Learning**_

<img src="img/maslow.png">

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.

In [ ]:
pd.read_csv('../vol/datasets/opening_df.csv').head()

Puedes mejorar considerablemente nuestra predicción?